In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##  Predicting Booster
Create a classification model that tells us whether or not universities mandate a booster *given* they already mandated a vaccine. Uses preprocessing from "Covid Model Creation" notebook. Note that this analysis better aligns with the iid assumption needed for machine learning--choosing dates to implement vaccine requirements is very much based on the action of other colleges. However, we can assume that choosing to implement a requirement is not *entirely* based on the actions of other institutions; once vaccination requirements seemed imminent, colleges evaluate their own situations and (likely) chose based on the science and social consequences for their students and surrounding environment, especially for a booster.

Ideally, I'd like to train a model to classify the universities that required the vaccine and those that didn't. Then, I would want to try a multi-level classification with three options: one for no mandate, one for a regular mandate, and one for a booster mandate. However, the lack of schools in the data without a vaccine mandate makes this analysis more difficult. Since my dataset is small, the model will likely overfit on those small examples and provide bad generalization. Still, I may try this after I finish with my booster analysis.

In [65]:
target_booster = pd.read_pickle('target_booster.pkl')
features_booster = pd.read_pickle('features_booster.pkl')
num_features = features_booster.shape[1]

In [66]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
categorical_preprocessor = OneHotEncoder(drop='first') # drop to avoid multicollinearity
numerical_preprocessor = StandardScaler() # normalize data to make it easier for sklearn models to handle

In [67]:
from sklearn.compose import ColumnTransformer # splits the column, transforms each subset differently, then concatenates
categorical_columns = ['ranking', 'Type', 'political_control_state', 'Region']
numerical_columns = list(set(features_booster.columns).difference(categorical_columns))
preprocessor = ColumnTransformer([('one-hot-encoder', categorical_preprocessor, categorical_columns),
                                  ('standard_scaler', numerical_preprocessor, numerical_columns)])

See distribution of values.

In [68]:
target_booster.value_counts()/target_booster.shape[0]

0    0.71223
1    0.28777
Name: booster, dtype: float64

Our dataset is moderately imbalanced. So, using normal methods for classification won't always be optimal. According to [Daskalaki, Kopanas, & Avouris (2006)](https://www.tandfonline.com/doi/full/10.1080/08839510500313653), with imbalanced data it's important to choose a reasonable metric to evaluate performance, adjust the dataset so that the minority class is adequately represented or seen in predictions, and select the best algorithm, sometimes combining them if a single one's performance is not good enough.

Specifically, we can change the cost of classifying minority/majority classes based on their prevalence and/or real-world implication, change the class distributions by undersampling the majority class or oversampling the minority class, use bagging or stacking to combine multiple models. I'll focus on changing the evaluation metric and the class weights in my models. I'll also use stratification to preserve the distribution of my dataset in training, testing, and cross-validation.

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_booster, target_booster, test_size=0.2, 
                                                    random_state=42, stratify=target_booster)

### Metrics
Accuracy is the fraction of cases identified correctly $= \frac{tp + tn}{tp + tn + fp + fn}$ 

Precision is the proportion of predicted positives that are true positives $= \frac{tp}{tp + fp}$ use to minimize fp

Recall is the proportion of true positives correctly identified $= \frac{tp}{tp + fn}$ use to minimize fn

F1 score is harmonic mean, but changes if confusion matrix flips.

Matthews Correlation Coefficient (MCC) robust to changes in confusion matrix--treats fp and fn the same.

ROC curve plots the true positive rate ($\frac{tp}{tp + fn}$) against the false positive rate ($\frac{fp}{fp + tn}$) according to the probability thresholds for classification. Take the area under the curve (AOC) to compare.

Precision/Recall curve plots precision against recall. Better than ROC curve if class imbalance exists. 

So, we have an imbalanced dataset, meaning I won't use regular accuracy.

Note that in previous attempts, I have just used accuracy as the metric to measure classification performance. This has resulted in very similar results to the dummy classifier; in some cases, the dummy classifier is better according to this metric. However, due to the class imbalance, I will use another metric. I want the positive examples (having a booster) to be classified as well--not just the negative examples as with the dummy classifier.

In [70]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
dclf = DummyClassifier(strategy='most_frequent')
dummy_scores = cross_val_score(dclf, X=X_train, y=y_train, cv=5)
print(f'Dummy | Outer accuracy: {dummy_scores.mean()} +/- {dummy_scores.std()}')

Dummy | Outer accuracy: 0.7118577075098814 +/- 0.019379684857124557


From [Boughorbel, Jarray, & El-Anbari](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0177678), MCC is shown to be "robust to data imbalance" along with AUC, but has a closed form which is easier to compute. MCC is related to the Pearson correlation coefficient, defined as $\frac{TP \times TN - FP \times FN}{\sqrt{(TP + FN)(TP + FP)(TN + FP)(TN+FN)}} \in [-1, 1]$ where 1 is perfect, -1 is inverse and 0 is average. Also, [Chicco, Tötsch, & Jurman (2021)](https://biodatamining.biomedcentral.com/articles/10.1186/s13040-021-00244-z) think MCC is generally the best single value evaluator of performance when *equal importance is given to both classes*. For example, if the goal is to minimize false positives, a different metric will be better. A high MCC 
> "means always high values for all the four basic rates of the confusion matrix: true
positive rate (TPR), true negative rate (TNR), positive predictive value (PPV), and negative predictive value (NPV)" 

Thus the other classification metrics will also be high. Note that bookmaker informedness (BM) should be used instead if classifiers are compared across datasets or if we are comparing the classifier to random guessing; however, here we are comparing with the same dataset and care about overall performance so MCC will be better and will be used in my analysis.

In [71]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import make_scorer
mcc_score = make_scorer(matthews_corrcoef, greater_is_better=True)

# test: should be 0 as no negatives predicted
dummy_scores_mcc = cross_val_score(dclf, X=X_train, y=y_train, cv=5, scoring=mcc_score)
print(f'Dummy | Outer MCC: {dummy_scores_mcc.mean()} +/- {dummy_scores_mcc.std()}')

Dummy | Outer MCC: 0.0 +/- 0.0


Additionally, I will use class weighting--treating it as a hyperparameter in grid search--to give more weight to the minority class. This will address the same problems as under/oversampling but not affect the overall distribution of the data. I'll use the same parameters in grid search for all of my models. Class 0 is more frequent, so weight class 1 more.

### Models
Now train models. Using [Sebastian Raschka's paper as a reference](https://arxiv.org/abs/1811.12808), especially his [code on nested CV which I took heavily from](https://github.com/rasbt/model-eval-article-supplementary/blob/master/code/nested_cv_code.ipynb). I initially did some selection with the model (you can see my previous commits on GitHub), but am going to have a more comprehensive approach going forward. Here's a summary of some of the things discussed in the paper:

Evaluate overall model performance:
- Use Monte-Carlo Cross-Validation
- Bootstrapping (LOOB) to ; use 50-200 samples
- 3 way holdout -- used in deep learning when dataset is large
- k-fold CV
    - can repeat many times (unnecessary for LOOCV), e.g., run 5-fold cross validation 100 times (with different random seeds), getting 500 test fold estimates
    - use LOOCV for small datasets--note it's approximately unbiased but with high variance
    - generally, increasing k decreases bias but increases variance and computation time

Hyperparameter tuning using CV:
- find best params using k-fold CV, then fit model with those params to entire training set to evaluate test set, afterwards using all data to fit final model
- feature selection could be done inside or outside the loop

Use Nested CV--outer loop estimates generalization error, inner loop selects model. For example, if we are doing 5-fold CV in the outer loop, we take the data from 4 folds, combine it into one dataset, then split that set into k folds and run CV. This will better account for the variance of the test set--same motivation as regular cross validation accounting for the variance of the validation set. Also note that in the previous example we select 5 best models and see the generalization error on each of them. We can choose one of those models or ensemble those models. See [Sergey Feldman's lecture](https://www.youtube.com/watch?v=DuDtXtKNpZs) for more.

**Figuring** out what to do after nested cross validation has been difficult. This is a summary of the previous resources and the stack overflow articles listed here ([1](https://stats.stackexchange.com/questions/232897/how-to-build-the-final-model-and-tune-probability-threshold-after-nested-cross-v/233027#233027), [2](https://stats.stackexchange.com/questions/65128/nested-cross-validation-for-model-selection/65156#65156), [3](https://stats.stackexchange.com/questions/341229/an-intuitive-understanding-of-each-fold-of-a-nested-cross-validation-for-paramet?rq=1), [4](https://stats.stackexchange.com/questions/244907/how-to-get-hyper-parameters-in-nested-cross-validation/245169#245169)):

My takeaways are that the inner loop is for model selection, while the outer loop is for generalization error. Therefore it would be wrong to choose a model based on the results of the outer loop. Instead, the estimates of the all the errors in outer loop for a specific model are averaged to provide the approximate generalization error for that model fitting method when all the data is used. After nested CV is done, apply your inner CV methods with all the data to select the optimal hyperparameters, using the results from nested CV as estimates for this procedure.
From 3:
>Thus: run the auto-tuning of hyperparameters on the whole data set just as you do during cross validation. Same hyperparameter combinations to consider, same strategy for selecting the optimum. In short: same training algorithm, just slightly different data (1/k additional cases).



Note that if the inner and outer estimates of the model are very different, this could signify overfitting. Also note if the inner estimates vary widely in their hyperparameters, this means the model is likely not stable. I will use repeated CV for both the inside and outside loops to promote stability. This means that the inner CV process will be repeated on each of the training sets separated by the outer CV process. Then, the outer CV process will be repeated as well to get a total of ```n_splits_outer*n_repeats_outer``` unbiased estimates. **look more into what this means**

In [72]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

# use 5-fold for inner and outer so there's enough data in validation set but not too much
n_splits_inner = 5
n_splits_outer = 5
n_repeats_inner = 5
n_repeats_outer = 5
inner_cv = RepeatedStratifiedKFold(n_splits=n_splits_inner, random_state=42, n_repeats=n_repeats_inner)
outer_cv = RepeatedStratifiedKFold(n_splits=n_splits_outer, random_state=42, n_repeats=n_repeats_outer)

Set up many models and grid search for each of them. After finding the best hyperparameters with the inner loop, train each model and evaluate on each of the k folds in the outer loop. Sklearn lets me do this nicely by passing in the grid search as a parameter for ```cross_val_score```. I will use random search, as it's more computationally efficient than normal grid search and sometimes better. See [Bergstra & Bengio 2012](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) for more; note they use neural networks but the general idea applies to all other models.

Create all the models I want to use and a parameter grid for each of them. I'll use logistic regression, random forest, and SVM. Nested CV should give 3\*k total estimates of model performance--k for each algorithm, where k is the number of folds in the outer CV. I'll average the estimates by model and report the standard deviation for each of the 3 models. Then, I'll choose the best model according to this and apply CV (the same as the inner CV used previously but with more data) with the same grid search parameters on the training set. Note that the generalization errors are no longer unbiased once I've chosen the minimum. Finally, I'll train those parameters on the entire training set, and use the test set to get an unbiased estimate.

In [73]:
from tqdm import tqdm # for timing
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

model_names = ['log', 'for', 'svm']

pipe_log = Pipeline([('pre', preprocessor), (model_names[0], LogisticRegression(random_state=42, solver='liblinear'))])
pipe_for = Pipeline([('pre', preprocessor), (model_names[1], RandomForestClassifier(random_state=42, oob_score=True))])
pipe_svm = Pipeline([('pre', preprocessor), (model_names[2], SVC(random_state=42))])
param_grid_log = [
    {'log__C': stats.loguniform(1e-5, 1e4), #[0.1**(x) for x in range(-4, 4)], # l2 default
     'log__class_weight': [{0:0.2, 1:0.8}, {0:0.33, 1:0.67}, {0:0.4, 1:0.6}, 'balanced', None]
    }
] 
param_grid_for = [
    {'for__max_features': [0.5, 0.75, 'sqrt', None], 
     'for__max_samples': [0.25, 0.5, 0.75, None],
     'for__min_samples_leaf': [1, 2, 5, 7],
     'for__max_leaf_nodes': [None, 2, 5, 10], 
     'for__max_depth': [10, 25, 50, None],
     'for__min_samples_split': [2, 3, 5, 7, 10],
     'for__class_weight': [{0:0.2, 1:0.8}, {0:0.33, 1:0.67}, {0:0.4, 1:0.6}, 'balanced', 'balanced_subsample', None]
    }
]
param_grid_svm = [ # see sources at bottom of page for reseaoning
    {'svm__kernel': ['linear', 'poly', 'rbf'],
     'svm__gamma': stats.loguniform(2**-15, 2**3), #[2**x for x in range(-12, 2, 2)],
     'svm__C': stats.loguniform(2**-5, 2**15), #[2**x for x in range(-4, 10, 2)],
     'svm__class_weight': [{0:0.2, 1:0.8}, {0:0.33, 1:0.67}, {0:0.4, 1:0.6}, 'balanced', None]
    }
]

validation_scores = np.zeros((3, n_splits_outer*n_repeats_outer))
for i, pipe, param_grid in tqdm(zip([0, 1, 2], [pipe_log, pipe_for, pipe_svm], [param_grid_log, param_grid_for, param_grid_svm])):
    name = list(pipe.named_steps.keys())[1]
    gcv = RandomizedSearchCV(pipe, param_grid, cv=inner_cv, scoring=mcc_score, n_iter=60)
    nested_score = cross_val_score(gcv, X=X_train, y=y_train, cv=outer_cv, scoring=mcc_score)
    validation_scores[i, :] = nested_score
    print(f'{name}: {nested_score}') # should output k_outer folds--the number of folds used in outer cv 

1it [09:01, 541.05s/it]

log: [ 0.03742406  0.29508445  0.30194054  0.41833001  0.1549826   0.11952286
  0.29730058  0.3125      0.30194054  0.24056261  0.38950871  0.18401748
  0.24056261  0.02405626 -0.25074294  0.25617377  0.18401748  0.3125
  0.19920477  0.3125      0.18232322  0.29277002  0.375       0.1549826
  0.08333333]


1it [48:29, 2909.02s/it]


KeyboardInterrupt: 

In [398]:
validation_scores_mean = validation_scores.mean(axis=1)
validation_scores_std = validation_scores.std(axis=1)
for i in range(3):
    print(f'{model_names[i]} | Outer MCC: {validation_scores_mean[i]} +/- {validation_scores_std[i]}')

log | Outer MCC: 0.2746063187634129 +/- 0.07990759656666674
for | Outer MCC: 0.3100452073613023 +/- 0.15534605172023658
svm | Outer MCC: 0.16842704605900796 +/- 0.24408354854208086


So, I have an approximately unbiased generalization error for all three model selection processes. Logistic regression and random forests have a similar estimated MCC and a similar standard deviation. SVM performs worse and has a similar standard deviation. Now run my model selection procedure on the entire training set.

In [400]:
final_gcv = RandomizedSearchCV(pipe_log, param_grid_log, cv=inner_cv, scoring=mcc_score, n_iter=60)
final_gcv.fit(X_train, y_train)
final_gcv.best_params_

{'log__C': 0.21335783405008515, 'log__class_weight': {0: 0.4, 1: 0.6}}

**Resampling (under/oversampling) or add more data if classifier not good enough**
[class weights?](https://stackoverflow.com/questions/30972029/how-does-the-class-weight-parameter-in-scikit-learn-work)

also, use bayesian optimization instead of grid search?

do repeated CV?

use stacking for the trained models?

## Final Evaluation and Training
Evaluate the chosen model on the test set, then train the model using all the data to get a final predictor.

In [ ]:
# y_pred = final_gcv.pred(X_test)
# matthews_corrcoef(y_test, y_pred)

Fit on entire data and save.

In [ ]:
# import joblib
# .fit(features_booster, target_booster)
# joblib.dump(, 'booster_log_model_jlib')

## Results & Takeaway
Create a web-page where people can input a state and zip-code and then our model can predict vaccine classification.

Also, create a histogram with number of predictions per class for all counties.

Then, create a map, using slider bars to indicate university-specific variables not specified by state or county. Shade each region differently based on their classification.

Use Dash to deploy.

**Note: After I deploy the booster model, I will do a multi-class prediction of no vaccine, vaccine, booster. This will require me to get more data (colleges and features) and research multi-class methods.**

## Sources (some in previous notebooks)
- Metrics for classification
    - https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226
    - https://www.youtube.com/watch?v=wpQiEHYkBys
    - https://www.youtube.com/watch?v=X9MZtvvQDR4 for imbalanced datasets
    - https://stackoverflow.com/questions/30972029/how-does-the-class-weight-parameter-in-scikit-learn-work class weights
    - https://stats.stackexchange.com/questions/391064/evaluating-classifiers-k-fold-cv-or-roc
    - https://www.kaggle.com/general/7517
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html#sklearn.metrics.matthews_corrcoef
- general tuning
    - https://towardsdatascience.com/why-is-the-log-uniform-distribution-useful-for-hyperparameter-tuning-63c8d331698
- random forest parameter tuning
    - https://stats.stackexchange.com/questions/344220/how-to-tune-hyperparameters-in-a-random-forest
    - https://arxiv.org/pdf/1804.03515.pdf
- svm parameter tuning
    - https://stats.stackexchange.com/questions/43943/which-search-range-for-determining-svm-optimal-c-and-gamma-parameters
    - https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf
        >"We found that trying exponentially growing sequences of C and γ is a
practical method to identify good parameters (for example, $C = 2^{−5}, 2^{−3}, . . . , 2^{15},γ = 2^{−15}, 2^{−13}, . . . , 2^{3}$)."
    - https://stats.stackexchange.com/questions/249881/svm-hyperparameters-tuning use bayesian optimization in future?